# Climate-economic risk decomposition notebook

This notebook makes the figure which compares the different price paths isolating a specific damage function.

In [ ]:
import sys
import datetime
import os 

import numpy as np
import matplotlib.pyplot as plt 
import matplotlib as mpl
import seaborn as sns

from src.analysis.output_unpacker import OutputUnpacker
from src.analysis.class_maker import make_class_instances
from src.tools import import_csv

# plotting parameters
color_list = ['#000000', '#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7'] * 2
marker_list = ['o', 's', 'P', '+', 'D', 'v', '3', 'm'] * 2
markersize = 6
linestyle_list = ['solid', 'dashed', 'dashdot', 'dotted'] * 4
linewidth = 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
cdds_params={'axes.linewidth': 3,
 'axes.axisbelow': False,
 'axes.edgecolor': 'black',
 'axes.facecolor': 'None',
 'axes.grid': False,
 'axes.labelcolor': 'black',
 'axes.spines.right': False,
 'axes.spines.top': False,
 'axes.titlesize': 20,
 'axes.labelsize': 20,
 'axes.titlelocation': 'left',
 'figure.facecolor': 'white',
 'figure.figsize': (18, 10),
 'lines.solid_capstyle': 'round',
 'lines.linewidth': 2.5,
 'patch.edgecolor': 'w',
 'patch.force_edgecolor': True,
 'text.color': 'black',
 'legend.frameon': False,
 'xtick.bottom': True,
 'xtick.major.width': 3,
 'xtick.major.size': 6,
 'xtick.color': 'black',
 'xtick.direction': 'out',
 'xtick.top': False,
 'ytick.color': 'black',
 'ytick.direction': 'out',
 'ytick.left': True,
 'ytick.right': False,
 'ytick.color' : 'black',
 'ytick.major.width': 3, 
 'ytick.major.size': 6,
 'axes.prop_cycle': plt.cycler(color=color_list, linestyle=linestyle_list),
 'font.size': 16,
 'font.family': 'serif'}
plt.rcParams.update(cdds_params)

node_times = [2020, 2030, 2060, 2100, 2150, 2200]
x_label = 'Year'

# make base filename
today = datetime.datetime.now()
year = str(today.year)
day = str(today.day)
month = str(today.month)

basefile = ''.join([month, '-', day, '-', year, '-'])

## Make list of runs to analyze

For any run we want to appear in the final figure, the run number should go here.

In [ ]:
runs = np.arange(0, 16, 1, dtype=int)
runs

## Make list of files of interest

Here we make lists of file prefixes (i.e., whatever the `name` parameter was when the simulation was executed) and paths. We take the list of paths and prefixes and make lists of filenames, which will be passed to `OutputUnpacker` later.

In [ ]:
# import BPW_research_runs.csv

data_csv_file = 'research_runs'
param_names, run_info, param_vals = import_csv(data_csv_file, delimiter=',', indices=2)
all_run_names = np.array([run_info[i][1] for i in range(len(run_info))])

# this an array of run names that are being analyzed
run_names = all_run_names[runs]
N_runs = len(run_names)

# path list (just data directory N_runs times)
paths = [os.getcwd()] * N_runs

# make list of full file names (path + run name + suffix) to be analyzed
output_list = [''.join([paths[file], run_names[file], "_node_period_output.csv"]) for file in range(N_runs)]
picklefile_list = [''.join([paths[file], run_names[file], "_log.pickle"]) for file in range(N_runs)]

## Make lists of necessary model classes

Make sure this matches the main file! To do this properly, the right classes with the right model parameters need to be initiated. 

In [ ]:
tree_list, damage_list, climate_list, emit_baseline_list = make_class_instances(runs)

## Use `OutputUnpacker` object to extract values from files

In [ ]:
# make one array of objects which correspond to the 'output' files (i.e., the files that end with
# _node_period_output.csv) and another for the pickle files
output_obj_list = [OutputUnpacker(output_list[file], run_names[file], 'output', 
                                    tree=tree_list[file], emit_baseline=emit_baseline_list[file],
                                    climate=climate_list[file], damage=damage_list[file]) for file in range(N_runs)]

pickle_obj_list = [OutputUnpacker(picklefile_list[file], run_names[file], 'pickle',
                                    tree=tree_list[file], emit_baseline=emit_baseline_list[file],
                                    climate=climate_list[file], damage=damage_list[file]) for file in range(N_runs)]

## Save fig?!

Turn on if you want the figure saved!

In [ ]:
save_fig = True
runs

## Import damage scatter points

In [ ]:
path = os.getcwd() + '/data/' # find current working directory

In [ ]:
burke_2049_ssp2 = np.genfromtxt(''.join([path, 'ssp2_2049_t_d.csv']), delimiter=',')
burke_2049_ssp2_t, burke_2049_ssp2_d = burke_2049_ssp2.T

burke_2099_ssp2 = np.genfromtxt(''.join([path, 'ssp2_2099_t_d.csv']), delimiter=',')
burke_2099_ssp2_t, burke_2099_ssp2_d = burke_2099_ssp2.T

struct_data = np.genfromtxt(''.join([path, "IPCC_WGII_RoseDamageEstimates.csv"]), 
                            delimiter=',', usecols=[0,1])
struct_T, struct_d = struct_data.T

hs_data = np.genfromtxt(''.join([path, "IPCC_WGII_HowardSternerDamageEstimates.csv"]), 
                        delimiter=',', usecols=[0,1])
hs_T, hs_d = hs_data.T

## Do fits

Basic idea: compute distributions for reach paramter, compute percentiles and means for plotting.

In [ ]:
gmst = np.arange(0, 7, 0.01)

In [ ]:
size = 1000
burke_mid_b2 = np.random.normal(loc=3.09e-3, scale=4.76e-4, 
                                size=size)
burke_mid_b1 = np.random.normal(loc=1.24e-2, scale=1.90e-3, 
                                size=size)

burke_end_b2 = np.random.normal(loc=-2.33e-3, scale=4.75e-4, 
                                size=size)
burke_end_b1 = np.random.normal(loc=7.21e-2, scale=1.47e-2, 
                                size=size)

struct_b2 = np.random.normal(loc=2.3e-3, scale=8.53e-4, 
                                size=size)
struct_b1 = np.random.normal(loc=2.05e-3, scale=7.59e-4, 
                                size=size)

meta_b2 = np.random.normal(loc=6.85e-3, scale=2.43e-3, 
                                size=size)
meta_b1 = np.random.normal(loc=2.98e-4, scale=1.06e-4, 
                                size=size)

In [ ]:
burke_mid_dam = gmst * (burke_mid_b2[:, None] * gmst + burke_mid_b1[:, None])
burke_end_dam = gmst * (burke_end_b2[:, None] * gmst + burke_end_b1[:, None])
struct_dam = gmst * (struct_b2[:, None] * gmst + struct_b1[:, None])
meta_dam = gmst * (meta_b2[:, None] * gmst + meta_b1[:, None])

In [ ]:
burke_mid_mean = np.mean(burke_mid_dam, axis=0)
burke_end_mean = np.mean(burke_end_dam, axis=0)
struct_mean = np.mean(struct_dam, axis=0)
meta_mean = np.mean(meta_dam, axis=0)

In [ ]:
burke_mid_66 = np.percentile(burke_mid_dam, 66, axis=0)
burke_mid_34 = np.percentile(burke_mid_dam, 34, axis=0)
burke_mid_95 = np.percentile(burke_mid_dam, 95, axis=0)
burke_mid_5 = np.percentile(burke_mid_dam, 5, axis=0)

burke_end_66 = np.percentile(burke_end_dam, 66, axis=0)
burke_end_34 = np.percentile(burke_end_dam, 34, axis=0)
burke_end_95 = np.percentile(burke_end_dam, 95, axis=0)
burke_end_5 = np.percentile(burke_end_dam, 5, axis=0)

struct_66 = np.percentile(struct_dam, 66, axis=0)
struct_34 = np.percentile(struct_dam, 34, axis=0)
struct_95 = np.percentile(struct_dam, 95, axis=0)
struct_5 = np.percentile(struct_dam, 5, axis=0)

meta_66 = np.percentile(meta_dam, 66, axis=0)
meta_34 = np.percentile(meta_dam, 34, axis=0)
meta_95 = np.percentile(meta_dam, 95, axis=0)
meta_5 = np.percentile(meta_dam, 5, axis=0)

## Tipping points

In [ ]:
tp_data = np.genfromtxt(''.join([path, "TP_data.csv"]), delimiter=',')

tp_t, tp_d = tp_data.T

B_MEAN = 0.48
A_MEAN = -0.04

CONSUMP_INIT = 61.88 # trillion 2020 USD (from World Bank)
GLOB_GDP_INIT = 84.75 # trillion 2020 USD (from World Bank)
factor = CONSUMP_INIT/GLOB_GDP_INIT

DRAWS = 10**4
b_dist = np.random.normal(loc=B_MEAN, scale=0.02, size=DRAWS)
a_dist = np.random.normal(loc=A_MEAN, scale=0.01, size=DRAWS)

#b_dist = np.random.random(size=DRAWS) * B_MEAN
#a_dist = np.random.random(size=DRAWS) * A_MEAN

gmst_tp = np.arange(0, 10, 0.01)
tp_dam = factor * (b_dist[:, None] * gmst_tp + a_dist[:, None] * gmst_tp**2) * 0.01
tp_dam_mean = (B_MEAN * gmst_tp + A_MEAN * gmst_tp**2) * factor * 0.01

In [ ]:
tp_66 = np.percentile(tp_dam, 66, axis=0)
tp_34 = np.percentile(tp_dam, 34, axis=0)
tp_95 = np.percentile(tp_dam, 95, axis=0)
tp_5 = np.percentile(tp_dam, 5, axis=0)

# Final figure

In [ ]:
import matplotlib.transforms as mtransforms

cost_row = ['A', 'B', 'C', 'D']
df_row = ['E', 'F', 'G', 'H']

fig, ax = plt.subplot_mosaic([cost_row, df_row], sharex=False,
                             gridspec_kw={'height_ratios': [1, 1], 'width_ratios': [1, 1, 1, 1]},
                            figsize=(16,9))

pref_lw = 3.5
feat_lw = 2
ax['A'].plot(node_times, np.mean(output_obj_list[4].price_path, axis=0), linestyle='solid', 
             label="2% discount rate", lw = pref_lw)
ax['A'].plot(node_times, np.mean(output_obj_list[5].price_path, axis=0), linestyle='dashed', 
             label="1.5% discount rate", lw = feat_lw)
ax['A'].plot(node_times, np.mean(output_obj_list[6].price_path, axis=0), linestyle='dashdot', 
             label="2.5% discount rate", lw = feat_lw)
ax['A'].plot(node_times, np.mean(output_obj_list[7].price_path, axis=0), linestyle='dotted', 
             label="3% discount rate", lw = feat_lw)
ax['A'].set_title("Statistical")
ax['A'].set_ylabel("Cost ($USD 2020)")

ax['B'].plot(node_times, np.mean(output_obj_list[8].price_path, axis=0), linestyle='solid', 
             label="2% discount rate", lw = pref_lw)
ax['B'].plot(node_times, np.mean(output_obj_list[9].price_path, axis=0), linestyle='dashed', 
             label="1.5% discount rate", lw = feat_lw)
ax['B'].plot(node_times, np.mean(output_obj_list[10].price_path, axis=0), linestyle='dashdot', 
             label="2.5% discount rate", lw = feat_lw)
ax['B'].plot(node_times, np.mean(output_obj_list[11].price_path, axis=0), linestyle='dotted', 
             label="3% discount rate", lw = feat_lw)
ax['B'].set_title("Structural")


ax['C'].plot(node_times, np.mean(output_obj_list[12].price_path, axis=0), linestyle='solid', 
             label="2% discount rate", lw = pref_lw)
ax['C'].plot(node_times, np.mean(output_obj_list[13].price_path, axis=0), linestyle='dashed', 
             label="1.5% discount rate", lw = feat_lw)
ax['C'].plot(node_times, np.mean(output_obj_list[14].price_path, axis=0), linestyle='dashdot', 
             label="2.5% discount rate", lw = feat_lw)
ax['C'].plot(node_times, np.mean(output_obj_list[15].price_path, axis=0), linestyle='dotted', 
             label="3% discount rate", lw = feat_lw)
ax['C'].set_title("Meta-analytic")


ax['D'].plot(node_times, np.mean(output_obj_list[1].price_path, axis=0), linestyle='dashed', 
             label="1.5% discount rate", lw = feat_lw, zorder=4, color=color_list[1])
ax['D'].plot(node_times, np.mean(output_obj_list[0].price_path, axis=0), linestyle='solid', 
             label="2% discount rate", lw = pref_lw, color='k')
ax['D'].plot(node_times, np.mean(output_obj_list[2].price_path, axis=0), linestyle='dashdot', 
             label="2.5% discount rate", lw = feat_lw, color=color_list[2])
ax['D'].plot(node_times, np.mean(output_obj_list[3].price_path, axis=0), linestyle='dotted', 
             label="3% discount rate", lw = feat_lw, color=color_list[3])
ax['D'].set_title("All sampled")

for i in cost_row:
    ax[i].set_ylim((0,350))
    ax[i].set_xticks([2020,2100,2200])
    ax[i].set_xticklabels(['2020', '2100', '2200'])
    ax[i].set_xlabel("Year")

ax['E'].plot(gmst, 100 * burke_mid_mean, linestyle='solid')
ax['E'].plot(gmst, 100 * burke_end_mean, linestyle='dashed', color='k')

ax['E'].scatter(burke_2049_ssp2_t, 100 * burke_2049_ssp2_d, alpha=0.5, marker='o', color='k', label='Mid-century')
ax['E'].scatter(burke_2099_ssp2_t, 100 * burke_2099_ssp2_d, alpha=0.5, marker='x', color='k', label='End-of-century')

ax['E'].fill_between(gmst, 100 * burke_mid_34, 100 * burke_mid_66, alpha=0.4, color=color_list[1])
ax['E'].fill_between(gmst, 100 * burke_mid_5, 100 * burke_mid_95, alpha=0.1, color=color_list[2])

ax['E'].fill_between(gmst, 100 * burke_end_34, 100 * burke_end_66, alpha=0.4, color=color_list[3])
ax['E'].fill_between(gmst, 100 * burke_end_5, 100 * burke_end_95, alpha=0.1, color=color_list[6])
ax['E'].set_ylabel("Damage (% GDP)")
ax['E'].legend(fontsize=13, loc='upper left')

ax['F'].plot(gmst, 100 * struct_mean)
ax['F'].scatter(struct_T, 100 * struct_d, alpha=0.5)
ax['F'].fill_between(gmst, struct_34 * 100, struct_66 * 100, alpha=0.4, color=color_list[1])
ax['F'].fill_between(gmst, struct_5 * 100, struct_95 * 100, alpha=0.1, color=color_list[2])

ax['G'].plot(gmst, meta_mean * 100)
ax['G'].scatter(hs_T, 100 * hs_d, alpha=0.5)
ax['G'].fill_between(gmst, meta_34 * 100, meta_66 * 100, alpha=0.4, color=color_list[1])
ax['G'].fill_between(gmst, meta_5 * 100, meta_95 * 100, alpha=0.1, color=color_list[2])

ax['H'].plot(gmst_tp, tp_dam_mean * 100)
ax['H'].scatter(tp_t, tp_d * 100, s=1.2, alpha=0.004)
ax['H'].fill_between(gmst_tp, tp_34 * 100, tp_66 * 100, alpha=0.5, color=color_list[1])
ax['H'].fill_between(gmst_tp, tp_5 * 100, tp_95 * 100, alpha=0.3, color=color_list[2])

ax['H'].set_xlabel("Temperature ($^{\circ}$C)")
ax['H'].set_xlim((0,10))
ax['H'].set_ylim((0,3.5))
ax['H'].set_yticks([0,1,2,3])
ax['H'].set_xticks([0,5,10])
    
for j in df_row[:-1]:
    ax[j].set_xlim((0,6))
    ax[j].set_ylim((0, 55))
    ax[j].set_xlabel("Temperature ($^{\circ}$C)")

ax['A'].set_yticks([0,100,200,300])
ax['B'].set_yticks([0,100,200,300])
ax['C'].set_yticks([0,100,200,300])
ax['D'].set_yticks([0,100,200,300])
ax['A'].set_ylim((0,320))
ax['B'].set_ylim((0,320))
ax['C'].set_ylim((0,320))
ax['D'].set_ylim((0,320))

sns.despine(offset=10, trim=True)
ax['D'].legend(loc='center', bbox_to_anchor=(-1.55, -0.355), ncol=5, fontsize=16)
fig.subplots_adjust(hspace=0.445, wspace=0.3)


# turn off y labels
turn_off_ylabels = ['B', 'C', 'D', 'F', 'G']
for i in turn_off_ylabels:
    ax[i].set_yticklabels([])

# label panels
for label in cost_row:
    # label physical distance in and down:
    trans = mtransforms.ScaledTranslation(2.15, 0.1, fig.dpi_scale_trans)
    ax[label].text(0.0, 1.0, label, transform=ax[label].transAxes + trans, fontsize=22, fontweight='bold',
            verticalalignment='top', bbox=dict(facecolor='1', edgecolor='none', pad=1))
    ax[label].tick_params(axis='both', labelsize=16)
    
for label in df_row:
    # label physical distance in and down:
    trans = mtransforms.ScaledTranslation(2.15, -0.4, fig.dpi_scale_trans)
    ax[label].text(0.0, 1.0, label, transform=ax[label].transAxes + trans, fontsize=22, fontweight='bold',
            verticalalignment='top', bbox=dict(facecolor='none', edgecolor='none', pad=1))
    ax[label].tick_params(axis='both', labelsize=16)
    

if save_fig:
    fig.savefig(''.join([basefile, 'clim-econ-risk-decomp.png']), bbox_inches='tight', dpi=400)